# Set-up

In [1]:
import os
from tqdm.autonotebook import tqdm
from pysradb.sraweb import SRAweb

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


/cellar/users/aklie/opt/miniconda3/envs/get_data/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  


In [2]:
# Choose the current dataset we are working with
dataset_name = "Chiou2021_islet_snATAC-seq"
srp_id = "SRP290255"

In [15]:
# Set-up directories
base_dir = "/cellar/users/aklie/data/datasets/Chiou2021_islet_snATAC-seq"
cwd = "/cellar/users/aklie/data/datasets/Chiou2021_islet_snATAC-seq/bin/data_acquisition"
fastq_dir = os.path.join(base_dir, "fastq")
os.chdir(cwd)
base_dir, cwd, fastq_dir

('/cellar/users/aklie/data/datasets/Chiou2021_islet_snATAC-seq',
 '/cellar/users/aklie/data/datasets/Chiou2021_islet_snATAC-seq/bin/data_acquisition',
 '/cellar/users/aklie/data/datasets/Chiou2021_islet_snATAC-seq/fastq')

In [4]:
# Connect to SRA
db = SRAweb()

# Get metadata

In [5]:
# Grab the metadata for the SRP
metadata = db.sra_metadata(srp_id, detailed=True)

In [11]:
# Save the metadata and the list of srr ids
metadata.to_csv(f"{srp_id}_metadata.tsv", index=False, sep="\t")
metadata["run_accession"].to_csv(f"{srp_id}_srr_ids.txt", index=False, header=False)

# Download (`sra` files)

In [12]:
db.download(df=metadata, out_dir=fastq_dir)

Checking download URLs
The following files will be downloaded: 

run_accession study_accession experiment_accession public_url                                                                                                  download_url                                                                                            out_dir                                                                                 filesize
SRR12957013   SRP290255       SRX9409841           https://sra-downloadb.be-md.ncbi.nlm.nih.gov/sos3/sra-pub-zq-24/SRR012/12957/SRR12957013/SRR12957013.lite.1 ftp://ftp-trace.ncbi.nlm.nih.gov/sra/sra-instant/reads/ByRun/sra/SRR/SRR129/SRR12957013/SRR12957013.sra /cellar/users/aklie/data/datasets/Chiou2021_islet_snATAC-seq/bin/data_acquisition/fastq 7.2 GB  
SRR12957014   SRP290255       SRX9409842           https://sra-downloadb.be-md.ncbi.nlm.nih.gov/sos3/sra-pub-zq-22/SRR012/12957/SRR12957014/SRR12957014.lite.1 ftp://ftp-trace.ncbi.nlm.nih.gov/sra/sra-instant/reads/B

SystemExit: 0

/cellar/users/aklie/opt/miniconda3/envs/get_data/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Convert to `fastq` files

In [ ]:
import glob
import subprocess

In [64]:
tmp_dir = "/cellar/users/aklie/tmp/fastq-dump"
gzip = True
split_files = True
threads = 4

In [72]:
# Loop through and print out each SRA download file within the subdirectories of the fastq_dir
# Run the following command parallel-fastq-dump --threads 4 --outdir . --split-files --tmpdir $tmp_dir --gzip -s SRR14048750.sra
for sra_file in glob.glob(os.path.join(fastq_dir, srp_id, "*", "*.sra")):
    sra_dir = os.path.dirname(sra_file)
    if gzip:
        cmd = f"parallel-fastq-dump --threads {threads} --outdir {sra_dir} --split-files --tmpdir {tmp_dir} --gzip -s {sra_file}"
    else:
        cmd = f"parallel-fastq-dump --threads {threads} --outdir {sra_dir} --split-files --tmpdir {tmp_dir} -s {sra_file}"
    print(cmd)
    
    # Check to see if the files have already been downloaded
    if len(glob.glob(os.path.join(sra_dir, "*.fastq*"))) > 0:
        print(f"Files already downloaded for {sra_dir}")
    else:
        subprocess.run(cmd, shell=True)

parallel-fastq-dump --threads 4 --outdir /cellar/users/aklie/data/igvf/beta_cell_networks/fastq/Wang2023_islet_snATAC-seq/SRP311849/SRX10424698 --split-files --tmpdir /cellar/users/aklie/tmp/fastq-dump --gzip -s /cellar/users/aklie/data/igvf/beta_cell_networks/fastq/Wang2023_islet_snATAC-seq/SRP311849/SRX10424698/SRR14048750.sra


In [65]:
# Loop through and print out each SRA download file within the subdirectories of the fastq_dir
for file in tqdm(os.listdir(fastq_dir)):
    if file.endswith(".sra"):
        file_path = os.path.join(fastq_dir, file)
        print(file_path)

100%|██████████| 1/1 [00:00<00:00, 13662.23it/s]


# DONE!

---

In [2]:
import pandas as pd

In [3]:
mdata = pd.read_csv("/cellar/users/aklie/data/datasets/Chiou2021_islet_snATAC-seq/bin/data_acquisition/SRP290255_metadata.tsv", sep="\t")
mdata

,run_accession,study_accession,study_title,experiment_accession,experiment_title,experiment_desc,organism_taxid,organism_name,library_name,library_strategy,...,sex,age,bmi,donor id,ena_fastq_http,ena_fastq_http_1,ena_fastq_http_2,ena_fastq_ftp,ena_fastq_ftp_1,ena_fastq_ftp_2
0,SRR12957013,SRP290255,Single-cell chromatin accessibility identifies...,SRX9409841,GSM4873768: Islet 1 snATAC (CB); Homo sapiens;...,GSM4873768: Islet 1 snATAC (CB); Homo sapiens;...,9606,Homo sapiens,NaN,ATAC-seq,...,male,32,32.3,UNOS:AFC2208,NaN,NaN,NaN,NaN,NaN,NaN
1,SRR12957014,SRP290255,Single-cell chromatin accessibility identifies...,SRX9409842,GSM4873769: Islet 2 snATAC (CB); Homo sapiens;...,GSM4873769: Islet 2 snATAC (CB); Homo sapiens;...,9606,Homo sapiens,NaN,ATAC-seq,...,male,45,29.3,UNOS:AFEA331,NaN,NaN,NaN,NaN,NaN,NaN
2,SRR12957015,SRP290255,Single-cell chromatin accessibility identifies...,SRX9409843,GSM4873770: Islet 3 snATAC (CB); Homo sapiens;...,GSM4873770: Islet 3 snATAC (CB); Homo sapiens;...,9606,Homo sapiens,NaN,ATAC-seq,...,male,62,36.1,UNOS:AFEP022,NaN,NaN,NaN,NaN,NaN,NaN
3,SRR12957016,SRP290255,Single-cell chromatin accessibility identifies...,SRX9409844,GSM4873771: Pancreas 1 snATAC (CB); Homo sapie...,GSM4873771: Pancreas 1 snATAC (CB); Homo sapie...,9606,Homo sapiens,NaN,ATAC-seq,...,male,33,30.9,nPOD:6004,NaN,NaN,NaN,NaN,NaN,NaN
4,SRR14135828,SRP290255,Single-cell chromatin accessibility identifies...,SRX9409845,GSM4873772: Pancreas 1 snATAC (10X); Homo sapi...,GSM4873772: Pancreas 1 snATAC (10X); Homo sapi...,9606,Homo sapiens,NaN,ATAC-seq,...,male,33,30.9,nPOD:6004,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
mdata

,run_accession,study_accession,study_title,experiment_accession,experiment_title,experiment_desc,organism_taxid,organism_name,library_name,library_strategy,...,sex,age,bmi,donor id,ena_fastq_http,ena_fastq_http_1,ena_fastq_http_2,ena_fastq_ftp,ena_fastq_ftp_1,ena_fastq_ftp_2
0,SRR12957013,SRP290255,Single-cell chromatin accessibility identifies...,SRX9409841,GSM4873768: Islet 1 snATAC (CB); Homo sapiens;...,GSM4873768: Islet 1 snATAC (CB); Homo sapiens;...,9606,Homo sapiens,NaN,ATAC-seq,...,male,32,32.3,UNOS:AFC2208,NaN,NaN,NaN,NaN,NaN,NaN
1,SRR12957014,SRP290255,Single-cell chromatin accessibility identifies...,SRX9409842,GSM4873769: Islet 2 snATAC (CB); Homo sapiens;...,GSM4873769: Islet 2 snATAC (CB); Homo sapiens;...,9606,Homo sapiens,NaN,ATAC-seq,...,male,45,29.3,UNOS:AFEA331,NaN,NaN,NaN,NaN,NaN,NaN
2,SRR12957015,SRP290255,Single-cell chromatin accessibility identifies...,SRX9409843,GSM4873770: Islet 3 snATAC (CB); Homo sapiens;...,GSM4873770: Islet 3 snATAC (CB); Homo sapiens;...,9606,Homo sapiens,NaN,ATAC-seq,...,male,62,36.1,UNOS:AFEP022,NaN,NaN,NaN,NaN,NaN,NaN
3,SRR12957016,SRP290255,Single-cell chromatin accessibility identifies...,SRX9409844,GSM4873771: Pancreas 1 snATAC (CB); Homo sapie...,GSM4873771: Pancreas 1 snATAC (CB); Homo sapie...,9606,Homo sapiens,NaN,ATAC-seq,...,male,33,30.9,nPOD:6004,NaN,NaN,NaN,NaN,NaN,NaN
4,SRR14135828,SRP290255,Single-cell chromatin accessibility identifies...,SRX9409845,GSM4873772: Pancreas 1 snATAC (10X); Homo sapi...,GSM4873772: Pancreas 1 snATAC (10X); Homo sapi...,9606,Homo sapiens,NaN,ATAC-seq,...,male,33,30.9,nPOD:6004,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# Grab stuff in between ":" and "snATAC", remove trailing and leading whitespace, and add "_" between any remaining whitespace
mdata.set_index("donor id")["experiment_title"].str.extract(r":\s*(.*?)\s*snATAC").iloc[:, 0].str.replace("\s+", "").to_csv("/cellar/users/aklie/data/datasets/Chiou2021_islet_snATAC-seq/ref/publication/id_mapping.txt")